In [1]:
import pyspark 
spark = pyspark.sql.SparkSession.builder.getOrCreate()
import pyspark.sql.functions as F

# Data Acquisition
**This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.**

Read the `case`, `department`, and `source` data into their own spark dataframes.

In [2]:
case = spark.read.csv('case.csv', header = True)
dept = spark.read.csv('dept.csv', header = True)
source = spark.read.csv('source.csv', header = True)
case.show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



### 1) Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [3]:
# source.write.csv('sources_csv.csv')
# source.write.json('sources_json.json')

- Inspect your folder structure. What do you notice?

> Well, it looks like it saves a bunch of random shit when all I asked for WAS A FUCKING CSV!

- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [4]:
case.show(1, vertical = True)
case.printSchema()

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: string (nullable = true)
 |-- case_closed

In [5]:
case = (case.withColumn('case_late', case.case_late.cast('boolean'))
            .withColumn('case_status', case.case_status.cast('boolean'))
            .withColumn('num_days_late', case.num_days_late.cast('float'))
            .withColumn('SLA_days', case.SLA_days.cast('float'))
            .withColumn('case_opened_date', F.to_timestamp(case.case_opened_date, 'M/d/yy H:mm'))
            .withColumn('case_closed_date', F.to_timestamp(case.case_closed_date, 'M/d/yy H:mm'))
            .withColumn('SLA_due_date', F.to_timestamp(case.SLA_due_date, 'M/mm/yy H:mm'))
       )
case.show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5088            
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | null                 
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [6]:
source.show(1, vertical = True)
source.printSchema()

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [7]:
dept.show(1, vertical = True)
dept.printSchema()

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [8]:
dept = dept.withColumn('dept_subject_to_SLA', dept.dept_subject_to_SLA.cast('boolean'))
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)



### 2) Explore

- How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [9]:
max_date = F.lit(case.select(F.max('case_closed_date')).head()[0])
min_date = F.lit(case.select(F.max('case_opened_date')).head()[0])
overdue_time = F.when(F.isnull('case_closed_date'), F.datediff(max_date, case.SLA_due_date))\
                .otherwise(F.datediff(case.case_closed_date  , case.SLA_due_date)).alias('overdue_time')
case.select('*', overdue_time).where(overdue_time == F.lit(case.select(F.max(overdue_time)).head()[0])).show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-01 08:30:00  
 case_late            | true                 
 num_days_late        | 348.64584            
 case_closed          | NO                   
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.7786              
 case_status          | null                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 overdue_time         | 584                  
only showing top 1 row



In [10]:
opened_time = F.when(F.isnull('case_closed_date'), F.datediff(max_date, case.case_opened_date))\
               .otherwise(F.datediff('case_closed_date','case_opened_date')).alias('case_lifetime')
case.select('*', opened_time).where(opened_time == F.lit(case.select(F.max(opened_time)).head()[0])).show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-01 08:30:00  
 case_late            | true                 
 num_days_late        | 348.64584            
 case_closed          | NO                   
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.7786              
 case_status          | null                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_lifetime        | 584                  
only showing top 1 row



- How many Stray Animal cases are there?

In [11]:
case.where(case.service_request_type.contains(F.lit('Stray'))).groupBy('service_request_type').count().show(100,truncate = False)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Stray Animal        |26760|
+--------------------+-----+



- How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [12]:
# case.groupBy('dept_division').count().show(100, truncate = False)
case.where(case.dept_division == 'Field Operations').groupBy('service_request_type').count().show()

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Animal Permits Re...| 3026|
|Injured Animal(Cr...| 9633|
|     Officer Standby| 3013|
|Animal Bite(Non-C...| 4750|
|        Stray Animal|26760|
|Trapped/Confined ...|11354|
|City Council Anim...|  365|
|Aggressive Animal...|24882|
|Animal Bite(Criti...|  708|
|Public Nuisance(O...|10715|
|Aggressive Animal...| 5266|
|Animal Cruelty(Cr...| 3001|
|Spay/Neuter Reque...|    1|
|      Animal Neglect|13441|
+--------------------+-----+



- Convert the council_district column to a string column.

In [13]:
#Already did that

- Extract the year from the case_closed_date column.

In [14]:
case.select(F.year('case_closed_date')).alias('closed_year').show(5)

+----------------------+
|year(case_closed_date)|
+----------------------+
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
+----------------------+
only showing top 5 rows



- Convert num_days_late from days to hours in new columns num_hours_late.

In [15]:
case.select((case.num_days_late * 24).alias('hours_late')).show(5)

+----------+
|hours_late|
+----------+
| -23964.21|
|  -48.3025|
| -72.53611|
|-360.27554|
|  8.931944|
+----------+
only showing top 5 rows



- Join the case data with the source and department data.

In [16]:
df = case.join(source, on = 'source_id', how = 'inner').join(dept, on = 'dept_division')
df.select('*').show(1, vertical = True)

-RECORD 0--------------------------------------
 dept_division          | Field Operations     
 source_id              | svcCRMLS             
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 SLA_due_date           | 2020-09-01 00:42:00  
 case_late              | false                
 num_days_late          | -998.5088            
 case_closed            | YES                  
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | null                 
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 source_username        | svcCRMLS             
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | true                 
only showing top 1 row



- Are there any cases that do not have a request source?

In [17]:
df.where(F.isnull(df.source_id)|(df.source_id == '')).show(vertical=True)

(0 rows)



- What are the top 10 service request types in terms of number of requests?

In [18]:
df.groupBy('service_request_type').count().sort(F.desc('count')).show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



- What are the top 10 service request types in terms of average days late?

In [19]:
df.groupBy('service_request_type').agg(F.mean(overdue_time).alias('avg_days_late')).sort(F.desc('avg_days_late')).show(10)

+--------------------+------------------+
|service_request_type|     avg_days_late|
+--------------------+------------------+
|Traffic Signal Gr...|             292.8|
|  Zoning: Junk Yards|211.47635135135135|
|Donation Containe...|178.04516129032257|
|Labeling for Used...|176.28571428571428|
|Record Keeping of...|167.14285714285714|
|Signage Requied f...|             163.5|
|Zoning: Recycle Yard|163.24747474747474|
|Storage of Used M...|           157.875|
|License Requied U...|             140.0|
|           Complaint|125.21570247933884|
+--------------------+------------------+
only showing top 10 rows



- Does number of days late depend on department?

In [20]:
df.groupBy('dept_division').agg(F.mean(overdue_time).alias('avg_days_late')).sort(F.desc('avg_days_late')).show(10)

+--------------------+------------------+
|       dept_division|     avg_days_late|
+--------------------+------------------+
|Code Enforcement ...|163.24747474747474|
|     311 Call Center|105.95015576323988|
|        Reservations|              83.5|
|Engineering Division|  81.1295633500358|
|Director's Office...| 61.74856046065259|
|               Shops| 31.65546218487395|
|           Tree Crew| 19.79310344827586|
|         Solid Waste|18.682598039215687|
|              Trades|18.476262933657942|
|Clean and Green N...|15.939024390243903|
+--------------------+------------------+
only showing top 10 rows



- How do number of days late depend on department and request type?

In [21]:
df.groupBy('dept_division', 'service_request_type').agg(F.mean(overdue_time).alias('avg_days_late')).sort(F.desc('avg_days_late')).show(10)

+--------------------+--------------------+------------------+
|       dept_division|service_request_type|     avg_days_late|
+--------------------+--------------------+------------------+
|             Signals|Traffic Signal Gr...|             292.8|
|    Code Enforcement|  Zoning: Junk Yards|211.47635135135135|
|    Code Enforcement|Donation Containe...|178.04516129032257|
|Code Enforcement ...|Labeling for Used...|176.28571428571428|
|Code Enforcement ...|Record Keeping of...|167.14285714285714|
|Code Enforcement ...|Signage Requied f...|             163.5|
|Code Enforcement ...|Zoning: Recycle Yard|163.24747474747474|
|Code Enforcement ...|Storage of Used M...|           157.875|
|Code Enforcement ...|        Bandit Signs|             149.6|
|Code Enforcement ...|License Requied U...|             140.0|
+--------------------+--------------------+------------------+
only showing top 10 rows

